# MOOD Validation using CMEMS In Situ Thematic Centre

This notebook aims to explotrethe the [CMEMS In Situ TAC Dashboard](http://www.marineinsitu.eu/dashboard/) with the goal of finding datasets for verification/validation in our hindcast/operational numerical models.

The code below was inspired by previous examples/trainings hosted at:
- https://github.com/CopernicusMarineInsitu
- https://github.com/DHI/watobs/blob/main/notebooks/CMEMS_in_situ_data.ipynb

## Functions and Packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

import os
from urllib import request
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import datetime
import xarray as xr
import ftputil
from collections import namedtuple
import numpy as np
from shapely.geometry import box, Point, Polygon
from geopy.distance import distance
import json
from urllib.parse import urlparse
import folium
from folium import plugins


In [4]:
def timeOverlap(row, targeted_range):
    # Checks if a file contains data in the specified time range (targeted_range)
    result = False
    try:
        date_format = "%Y-%m-%dT%H:%M:%SZ"
        targeted_ini = datetime.datetime.strptime(targeted_range.split('/')[0], date_format)
        targeted_end = datetime.datetime.strptime(targeted_range.split('/')[1], date_format)
        time_start = datetime.datetime.strptime(row['time_coverage_start'],date_format)
        time_end = datetime.datetime.strptime(row['time_coverage_end'],date_format)
        Range = namedtuple('Range', ['start', 'end'])
        r1 = Range(start=targeted_ini, end=targeted_end)
        r2 = Range(start=time_start, end=time_end)
        latest_start = max(r1.start, r2.start)
        earliest_end = min(r1.end, r2.end)
        delta = (earliest_end - latest_start).days + 1
        overlap = max(0, delta)
        if overlap != 0:
            result = True
    except Exception as e:
        pass
    return result

In [5]:
def spatialOverlap(row, targeted_bbox):
    # Checks if a file contains data in the specified area (targeted_bbox)
    result = False
    try:
        geospatial_lat_min = float(row['geospatial_lat_min'])
        geospatial_lat_max = float(row['geospatial_lat_max'])
        geospatial_lon_min = float(row['geospatial_lon_min'])
        geospatial_lon_max = float(row['geospatial_lon_max'])
        targeted_bounding_box = box(targeted_bbox[0], targeted_bbox[1],targeted_bbox[2], targeted_bbox[3])
        bounding_box = box(geospatial_lon_min, geospatial_lat_min,geospatial_lon_max, geospatial_lat_max)
        if targeted_bounding_box.intersects(bounding_box):  # check other rules on https://shapely.readthedocs.io/en/stable/manual.html
            result = True
    except Exception as e:
        pass
    return result

In [6]:
def lastLocationInRange(row, targeted_bbox):
    # Checks if a file has been produced by a platform whose last position is within the specified area (targeted_bbox)
    result = False
    try:
        geospatial_lat = float(row['last_latitude_observation'])
        geospatial_lon = float(row['last_longitude_observation'])
        targeted_bounding_box = box(targeted_bbox[0], targeted_bbox[1],targeted_bbox[2], targeted_bbox[3])
        location = Point(geospatial_lon, geospatial_lat)
        if targeted_bounding_box.contains(location):#check other rules on https://shapely.readthedocs.io/en/stable/manual.html
            result = True
    except Exception as e:
        pass
    return result

In [7]:
def getIndexFiles(usr,pas,dataset):
    # Provides the index files available at the ftp server
    indexes = dataset['index_files'] + [dataset['index_platform']]
    with ftputil.FTPHost(dataset['host'], usr, pas) as ftp_host:  # connect to CMEMS FTP
        for index in indexes:
            remotefile= "/".join(['Core',dataset['product'],dataset['name'],index])
            print('.....Downloading ' + index)
            localfile = os.path.join(os.getcwd(),'data','index_files',index)
            ftp_host.download(remotefile,localfile)  # remote, local
    print('Ready!')

In [8]:
def getIndexFilesInfo(usr, pas, dataset, targeted_bbox):
    # Load and merge in a single entity all the information contained on each file descriptor of a given dataset
    # 1) Loading the index platform info as dataframe
    path2file = os.path.join(os.path.join(os.path.join(os.getcwd(),'data'), 'index_files'), dataset['index_platform'])
    indexPlatform = readIndexFileFromCWD(path2file, None)
    indexPlatform.rename(columns={indexPlatform.columns[0]: "platform_code" }, inplace = True)
    indexPlatform = indexPlatform.drop_duplicates(subset='platform_code', keep="first")
    # 2) Loading the index files info as dataframes
    netcdf_collections = []
    for filename in dataset['index_files']:
        path2file = os.path.join(os.getcwd(),'data', 'index_files',filename)
        indexFile = readIndexFileFromCWD(path2file, targeted_bbox) # replaced from targeted_bbox
        netcdf_collections.append(indexFile)
    netcdf_collections = pd.concat(netcdf_collections)
    # 3) creating new columns: derived info
    netcdf_collections['netcdf'] = netcdf_collections['file_name'].str.split('/').str[-1]
    netcdf_collections['file_type'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[1]
    netcdf_collections['data_type'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[2]
    netcdf_collections['platform_code'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[3]
    # 4) Merging the information of all files
    headers = ['platform_code','wmo_platform_code', 'institution_edmo_code', 'last_latitude_observation', 'last_longitude_observation','last_date_observation']
    result = pd.merge(netcdf_collections,indexPlatform[headers],on='platform_code')
    print('Ready!')
    return result

In [9]:
def readIndexFileFromCWD(path2file, targeted_bbox):
    #Load as pandas dataframe the file in the provided path
    filename = os.path.basename(path2file)
    print('...Loading info from: '+filename)
    if targeted_bbox != None:
        raw_index_info =[]
        chunks = pd.read_csv(path2file, skiprows=5,chunksize=1000)
        for chunk in chunks:
            chunk['spatialOverlap'] = chunk.apply(spatialOverlap,targeted_bbox=targeted_bbox,axis=1)
            raw_index_info.append(chunk[chunk['spatialOverlap'] == True])
        return pd.concat(raw_index_info)
    else:
        result = pd.read_csv(path2file, skiprows=5)
        try:
            result = result.rename(columns={"provider_edmo_code": "institution_edmo_code"})
        except Exception as e:
            pass
        return result

In [10]:
def closest_distances(ref_points, geojson_file):
    # Load the geojson file containing all coastlines and islands
    with open(geojson_file) as f:
        data = json.load(f)

    # Initialize list of closest distances
    closest_dists = []

    # Loop over all reference points
    for i, ref_point in enumerate(ref_points):
        # Initialize minimum distance to a large value
        min_dist = float('inf')

        # Loop over all polygons in the geojson file to calculate the nearest point of each polygon to the reference point
        for feature in data['features']:
            coords = feature['geometry']['coordinates']
            if feature['geometry']['type'] == 'Polygon':
                poly = Polygon(coords[0])
            elif feature['geometry']['type'] == 'MultiPolygon':
                poly = Polygon(coords[0][0])
                for p in coords[0][1:]:
                    poly = poly.union(Polygon(p))
            else:
                raise ValueError('Unknown geometry type: {}'.format(feature['geometry']['type']))

            nearest_point = poly.exterior.interpolate(poly.exterior.project(Point(ref_point[1], ref_point[0])))

            # Compute the distance between the reference point and the nearest point of the polygon using the haversine formula
            dist = distance((ref_point[0], ref_point[1]), (nearest_point.y, nearest_point.x)).km

            # Update the minimum distance
            if dist < min_dist:
                min_dist = dist

        # Add the closest distance to the list
        closest_dists.append(min_dist)

    # Return the list of closest distances
    return closest_dists

## Data access

In [11]:
# Insert here your user and password
with open('c:\\Users\\psan\\OneDrive - DHI\\Documents\\1.Projects\\GASP\\Wave data task\\CMEMS_credentials.txt','r') as f:
    lines=f.readlines()
    user = lines[1].strip() #type CMEMS user name
    password = lines[2].strip() #type CMEMS password

In [12]:
# product_name = 'INSITU_MED_NRT_OBSERVATIONS_013_035' #type aimed In Situ product 
product_name = '/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/cmems_obs-ins_glo_phybgcwav_mynrt_na_irr' #type aimed In Situ product 
host = 'nrt.cmems-du.eu' #type aimed host (nrt.cmems-du.eu or my.cmems-du)
index_file = 'index_latest.txt' #type aimed index file 

In [13]:
dataset = {
    'host': 'nrt.cmems-du.eu',#ftp host => nrt.cmems-du.eu for Near Real Time products
    'product': 'INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030',#name of the In Situ Near Real Time product in the GLO area
    'name': 'cmems_obs-ins_glo_phybgcwav_mynrt_na_irr',#name of the dataset available in the above In Situ Near Real Time product
    #'index_files': ['index_latest.txt', 'index_monthly.txt', 'index_history.txt'],#files describing the content of the lastest, monthly and history netCDF file collections available withint he above dataset
    'index_files': ['index_latest.txt', 'index_history.txt'],#files describing the content of the lastest, monthly and history netCDF file collections available withint he above dataset
    'index_platform': 'index_platform.txt',#files describing the netwotk of platforms contributting with files in the abve collections
}

### Specify a region (bounding box) [Optional]

In [33]:
 #selection criteria: spatial coverage >> 
targeted_geospatial_lat_min = 32   # enter min latitude of your bounding box
targeted_geospatial_lat_max =  52   # enter max latitude of your bounding box
targeted_geospatial_lon_min = -131  # enter min longitude of your bounding box
targeted_geospatial_lon_max =  -115  # enter max longitude of your bounding box  

targeted_bounding_box2 = box(targeted_geospatial_lon_min, targeted_geospatial_lat_min, targeted_geospatial_lon_max, targeted_geospatial_lat_max)
targeted_bounding_box = [targeted_geospatial_lon_min, targeted_geospatial_lat_min, targeted_geospatial_lon_max, targeted_geospatial_lat_max]  # (minx, miny, maxx, maxy)


In [34]:
m = folium.Map(location=[53.0, 0], zoom_start=4)
upper_left = [targeted_geospatial_lat_max, targeted_geospatial_lon_min]
upper_right = [targeted_geospatial_lat_max, targeted_geospatial_lon_max]
lower_right = [targeted_geospatial_lat_min, targeted_geospatial_lon_max]
lower_left = [targeted_geospatial_lat_min, targeted_geospatial_lon_min]
edges = [upper_left, upper_right, lower_right, lower_left]
polygon = folium.vector_layers.Polygon(locations=edges)
m.add_child(polygon)
m.fit_bounds(polygon.get_bounds())
m

### 1.2. Download database indexes

In [35]:
getIndexFiles(user,password,dataset) # skip if already downloaded, ~50s

.....Downloading index_latest.txt
.....Downloading index_history.txt
.....Downloading index_platform.txt
Ready!


In [36]:
info = getIndexFilesInfo(user, password, dataset, targeted_bounding_box) # Add None to the last function entry if you want to search the entire world.
#info = getIndexFilesInfo(user, password, dataset, None) # Add None to the last function entry if you want to search the entire world.
info.transpose()

...Loading info from: index_platform.txt
...Loading info from: index_latest.txt
...Loading info from: index_history.txt
Ready!


,0,1,2,3,4,5,6,7,8,9,...,9671,9672,9673,9674,9675,9676,9677,9678,9679,9680
# product_id,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,...,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...
geospatial_lat_min,32.65247,32.57725,32.49455,32.4029,32.26543,32.08133,31.93445,31.93105,32.0276,32.11235,...,-16.93333,-65.2167,44.03333,43.71667,-39.28333,32.59665,11.08056,44.59383,19.0595,46.141
geospatial_lat_max,32.7339,32.64695,32.56485,32.48353,32.38853,32.24473,32.0621,32.01503,32.1014,32.1483,...,43.7,60.45,45.16667,44.61667,45.0,69.03702,46.07487,64.00617,53.99867,46.141
geospatial_lon_min,-119.59353,-119.77603,-119.95038,-120.13867,-120.39708,-120.75878,-121.07158,-121.0506,-120.8491,-120.68658,...,-91.98333,-81.5667,-124.68333,-124.6,144.55,-169.24037,141.32722,177.82899,-123.2,-124.574
geospatial_lon_max,-119.42292,-119.62008,-119.7963,-119.97068,-120.16483,-120.4381,-120.79755,-120.87118,-120.70358,-120.60648,...,-151.35001,-151.75,-124.01667,-124.03333,-117.71667,-117.22478,-124.34647,-122.23798,14.7295,-124.574
time_coverage_start,2023-05-22T02:03:15Z,2023-05-23T02:34:15Z,2023-05-24T02:12:45Z,2023-05-25T01:42:45Z,2023-05-26T01:15:00Z,2023-05-27T00:43:45Z,2023-05-28T02:49:45Z,2023-06-15T00:27:45Z,2023-06-16T00:06:15Z,2023-06-17T00:08:00Z,...,2009-03-27T18:00:00Z,2010-11-01T18:00:00Z,2012-01-06T22:00:00Z,2011-12-07T08:00:00Z,2011-11-14T22:56:00Z,2017-09-25T18:48:42Z,2008-10-28T02:01:42Z,2008-02-07T15:59:43Z,2002-05-21T03:16:12Z,2017-05-05T13:00:00Z
time_coverage_end,2023-05-22T23:58:45Z,2023-05-23T23:18:30Z,2023-05-24T22:45:00Z,2023-05-25T22:17:30Z,2023-05-26T21:46:15Z,2023-05-27T23:57:00Z,2023-05-28T23:22:15Z,2023-06-15T21:09:00Z,2023-06-16T21:09:30Z,2023-06-17T21:04:30Z,...,2012-10-09T18:00:00Z,2012-03-21T02:00:00Z,2012-02-03T08:00:00Z,2012-01-06T17:00:00Z,2012-03-24T08:27:00Z,2021-11-17T23:56:24Z,2021-07-27T04:07:57Z,2023-05-22T18:52:46Z,2013-10-07T15:29:43Z,2021-12-01T21:40:00Z
institution,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,...,Unknown institution,Unknown institution,Unknown institution,Unknown institution,China (Unknown),United states (Unknown),US DOC NOAA NMFS HONOLULU,United states (Unknown),United Kingdom (Unknown),National Data Buoy Center (NDBC) National Ocea...
date_update,2023-05-23T09:01:06Z,2023-05-24T05:01:06Z,2023-05-25T05:01:07Z,2023-05-26T05:01:06Z,2023-05-27T05:01:06Z,2023-05-2

## Data Selection

### Create a subset based on selected criteria

In [37]:
# Choose a colection
targeted_collection = 'history' #try 'latest', 'monthly' or 'history'
# Choose a data type (MO=fixed buoys, mooring time series, fixed observations)
targeted_data_type1 = 'MO' # try others: TG for Tide Gauges, PF for profilers etc =>Product User Manual: https://archimer.ifremer.fr/doc/00324/43494/
targeted_data_type2 = 'TX' # try others: TG for Tide Gauges, PF for profilers etc =>Product User Manual: https://archimer.ifremer.fr/doc/00324/43494/
# Choose a file type
targeted_file_type = 'PR' # try others: PR for profiles, TS for time series etc =>Product User Manual: https://archimer.ifremer.fr/doc/00324/43494/
# Choose parameters of choice
# SLEV=Water surface height above a specific datum
# HCSP=Horizontal current speed, HCDT=Current to direction relative true north
# VHM0=Spectral significant wave height (Hm0), VTPK=Wave period at spectral peak / peak period (Tp)
# VGHS=Generic significant wave height (Hs)
# VTM02=Spectral moments (0,2) wave period (Tm02)
#WSPD =	Horizontal wind speed, WDIR = Wind from direction relative true north
targeted_parameter1= 'HCSP'
targeted_parameter2= 'HCDT'
targeted_parameter3= 'VTM02'

# Check if the station is live (criteria is last_date_observation at least 3 days ago)
# The history collection can have more stations than the latest, using this criteria

# Declare all the necessary conditions
#condition0 = info['file_name'].str.contains(targeted_collection)
#condition01 = info['data_type'].str.contains(targeted_data_type1)
#condition011 = info['data_type'].str.contains(targeted_data_type2)
#condition02 = info['file_type'].str.contains(targeted_file_type)
#condition1 = info['parameters'].str.contains(targeted_parameter1)
#condition2 = info['parameters'].str.contains(targeted_parameter2)
#condition3 = info['parameters'].str.contains(targeted_parameter3)
# Apply conditions
#subset=info[condition0 & (condition01 | condition011) & condition02 & condition1 & condition2]
#subset=info[condition0 & condition01 &condition1 & condition2 & condition3]
#subset=info[condition0 & condition01 &condition1 & condition2]
#subset=info[condition0 & condition1 & condition2]



In [38]:
# Criteria for water level data
subset_water_level=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                        info['data_type'].str.contains('TG') & # Select only tidal gauges
                        info['parameters'].str.contains('SLEV') # Select SLEV=Water surface height above a specific datum
                        ]
# Criteria for current profiles
subset_current_profiles=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                                    (info['data_type'].str.contains('MO') | info['data_type'].str.contains('TX')) & # Select mooring bouys and thermistor chains
                                    info['file_type'].str.contains('PR') & # Select only profiles
                                    info['parameters'].str.contains('HCSP') & info['parameters'].str.contains('HCDT') # Select current speed and direction
                                    ]                 
# Criteria for current depth-averaged
subset_current_depth_averaged=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                                    info['data_type'].str.contains('MO') & # Select mooring bouys
                                    info['file_type'].str.contains('TS') & # Select only time series
                                    info['parameters'].str.contains('HCSP') & info['parameters'].str.contains('HCDT') # Select current speed and direction
                                    ]
# Criteria for waves Hm0 and Tp
subset_waves_Hm0_Tp=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                            info['data_type'].str.contains('MO') & # Select mooring bouys 
                            info['file_type'].str.contains('TS') & # Select only time series
                            info['parameters'].str.contains('VHM0') & info['parameters'].str.contains('VTPK') # Select Hm0 and Tp
                            ]
# Criteria for waves Hm0 and Tm02
subset_waves_Hm0_Tm02=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                            info['data_type'].str.contains('MO') & # Select mooring bouys 
                            info['file_type'].str.contains('TS') & # Select only time series
                            info['parameters'].str.contains('VHM0') & info['parameters'].str.contains('VTM02') # Select Hm0 and Tm02
                            ]
# Criteria for wind speed and direction
subset_winds=info[info['file_name'].str.contains('history') & # select history index files, which also contains latest
                    info['data_type'].str.contains('MO') & # Select mooring bouys and thermistor chains
                    info['file_type'].str.contains('TS') & # Select only time series
                    info['parameters'].str.contains('WSPD') & info['parameters'].str.contains('WDIR') # Select current speed and direction
                    ]


### Select and plot a subset

In [39]:
subset1=subset_waves_Hm0_Tp
subset1.transpose()

,1361,1467,1529,1590,1652,1713,1774,1835,1897,1958,...,9596,9597,9598,9599,9600,9601,9611,9627,9628,9629
# product_id,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,...,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...
geospatial_lat_min,42.57,34.868,38.228,39.196,33.739,37.75,41.84,35.703,46.12,47.336,...,34.439,43.585,43.726,33.704,39.368,32.926,48.9908,48.333,48.433,48.836
geospatial_lat_max,42.664,35.0,38.253,39.235,33.763,37.759,41.852,35.774,46.163,47.353,...,34.439,43.586,43.76,33.704,39.368,32.926,49.037,48.333,48.433,48.898
geospatial_lon_min,-130.537,-121.019,-123.325,-123.974,-119.08,-122.839,-124.386,-121.905,-124.514,-124.75,...,-120.766,-124.29,-124.225,-119.004,-123.91,-117.277,-125.8,-123.55,-123.45,-126.069
geospatial_lon_max,-130.36,-120.857,-123.301,-123.969,-119.044,-122.82,-124.38,-121.857,-124.485,-124.704,...,-120.766,-124.29,-124.225,-119.004,-123.91,-117.277,-125.744,-123.55,-123.45,-126.013
time_coverage_start,1975-07-17T02:00:00Z,1980-10-07T18:00:00Z,1981-04-01T00:00:00Z,1981-04-01T19:00:00Z,1982-04-20T20:00:00Z,1982-07-23T16:00:00Z,1983-09-01T00:00:00Z,1983-12-01T22:00:00Z,1984-03-27T23:00:00Z,1987-06-09T17:00:00Z,...,2015-07-10T14:00:00Z,2017-04-05T16:30:00Z,2017-04-05T16:30:00Z,2017-09-24T16:00:00Z,2017-09-28T20:00:00Z,2021-11-12T17:26:00Z,1970-06-26T23:00:00Z,1990-11-27T20:50:00Z,1992-10-29T17:50:00Z,2012-10-31T00:19:00Z
time_coverage_end,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,...,2017-07-24T18:30:00Z,2018-04-20T21:30:00Z,2018-04-20T21:30:00Z,2018-11-20T17:30:00Z,2018-09-26T10:30:00Z,2022-03-09T19:26:00Z,1998-06-17T08:50:00Z,1991-04-30T08:50:00Z,1994-01-05T16:20:00Z,2013-04-28T21:45:00Z
institution,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,...,Scripps Institution of Oceanography La Jolla (...,National Renewable Energy Laboratory,National Renewable Energy Laboratory,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Fisheries and Oceans Canada Marine Environment...,Environment and Climate Change Canada,Environment and Climate Change Canada,Fisheries and Oceans Canada
date_update,2023-06-03T17:45:21Z,2023-06-03

In [40]:
# Clear the map and draw the search area defined above
m = folium.Map(location=[53.0, 0], zoom_start=4)
upper_left = [targeted_geospatial_lat_max, targeted_geospatial_lon_min]
upper_right = [targeted_geospatial_lat_max, targeted_geospatial_lon_max]
lower_right = [targeted_geospatial_lat_min, targeted_geospatial_lon_max]
lower_left = [targeted_geospatial_lat_min, targeted_geospatial_lon_min]
edges = [upper_left, upper_right, lower_right, lower_left]
polygon = folium.vector_layers.Polygon(locations=edges)
m.add_child(polygon)
m.fit_bounds(polygon.get_bounds())
# Plot the selected stations based on the subset selected above
for ind in subset1.index:
    legend='First measurement: '+subset1['time_coverage_start'][ind]+' Last measurement: '+subset1['last_date_observation'][ind]+' Platform code:'+subset1['platform_code'][ind]+' Parameters:'+subset1['parameters'][ind]
    folium.Marker(location=[subset1['last_latitude_observation'][ind],subset1['last_longitude_observation'][ind]],
                  popup=legend,
                  icon=folium.Icon(color='blue')).add_to(m)
# Show map
m


### Filter the subset by distance to the coastline [Optional]

In [41]:
# Query a list with tuples of (latitude,longitute) for all stations in the above subset
sublat=subset1['last_latitude_observation']
sublon=subset1['last_longitude_observation']
ref_points=list(zip(sublat,sublon))


In [42]:
# Load a GEOJSON file with the world's coastlines. Downloaded from https://geojson-maps.ash.ms/
geojson_file="c:\\Users\\psan\\OneDrive - DHI\Documents\\1.Projects\\GASP\Wave data task\\custom.geojson"

In [43]:
# Compute the distance to the coastline for each file on subset
# TO-DO: find unique (lat,lon) and only check by location
closest_dists = closest_distances(ref_points, geojson_file)
subset1['coastline_distance']=closest_dists
subset1.transpose()

,1361,1467,1529,1590,1652,1713,1774,1835,1897,1958,...,9596,9597,9598,9599,9600,9601,9611,9627,9628,9629
# product_id,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,...,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...
geospatial_lat_min,42.57,34.868,38.228,39.196,33.739,37.75,41.84,35.703,46.12,47.336,...,34.439,43.585,43.726,33.704,39.368,32.926,48.9908,48.333,48.433,48.836
geospatial_lat_max,42.664,35.0,38.253,39.235,33.763,37.759,41.852,35.774,46.163,47.353,...,34.439,43.586,43.76,33.704,39.368,32.926,49.037,48.333,48.433,48.898
geospatial_lon_min,-130.537,-121.019,-123.325,-123.974,-119.08,-122.839,-124.386,-121.905,-124.514,-124.75,...,-120.766,-124.29,-124.225,-119.004,-123.91,-117.277,-125.8,-123.55,-123.45,-126.069
geospatial_lon_max,-130.36,-120.857,-123.301,-123.969,-119.044,-122.82,-124.38,-121.857,-124.485,-124.704,...,-120.766,-124.29,-124.225,-119.004,-123.91,-117.277,-125.744,-123.55,-123.45,-126.013
time_coverage_start,1975-07-17T02:00:00Z,1980-10-07T18:00:00Z,1981-04-01T00:00:00Z,1981-04-01T19:00:00Z,1982-04-20T20:00:00Z,1982-07-23T16:00:00Z,1983-09-01T00:00:00Z,1983-12-01T22:00:00Z,1984-03-27T23:00:00Z,1987-06-09T17:00:00Z,...,2015-07-10T14:00:00Z,2017-04-05T16:30:00Z,2017-04-05T16:30:00Z,2017-09-24T16:00:00Z,2017-09-28T20:00:00Z,2021-11-12T17:26:00Z,1970-06-26T23:00:00Z,1990-11-27T20:50:00Z,1992-10-29T17:50:00Z,2012-10-31T00:19:00Z
time_coverage_end,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,...,2017-07-24T18:30:00Z,2018-04-20T21:30:00Z,2018-04-20T21:30:00Z,2018-11-20T17:30:00Z,2018-09-26T10:30:00Z,2022-03-09T19:26:00Z,1998-06-17T08:50:00Z,1991-04-30T08:50:00Z,1994-01-05T16:20:00Z,2013-04-28T21:45:00Z
institution,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,...,Scripps Institution of Oceanography La Jolla (...,National Renewable Energy Laboratory,National Renewable Energy Laboratory,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Fisheries and Oceans Canada Marine Environment...,Environment and Climate Change Canada,Environment and Climate Change Canada,Fisheries and Oceans Canada
date_update,2023-06-03T17:45:21Z,2023-06-03

In [44]:
# Let's choose a threshold for the coastline distance
coast_min_dist=15 # This is based on the 15km and 30km grid cell sizes for the regional and global models, respectively
condition_dist = subset1['coastline_distance']>coast_min_dist

In [45]:
subset2=subset1[condition_dist]
subset2.transpose()

,1361,1467,1529,1590,1652,1713,1835,1897,1958,2019,...,9591,9592,9594,9595,9596,9599,9611,9627,9628,9629
# product_id,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,...,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...
geospatial_lat_min,42.57,34.868,38.228,39.196,33.739,37.75,35.703,46.12,47.336,36.75,...,37.753,33.821,33.953,33.4,34.439,33.704,48.9908,48.333,48.433,48.836
geospatial_lat_max,42.664,35.0,38.253,39.235,33.763,37.759,35.774,46.163,47.353,36.791,...,37.753,33.821,33.953,33.4,34.439,33.704,49.037,48.333,48.433,48.898
geospatial_lon_min,-130.537,-121.019,-123.325,-123.974,-119.08,-122.839,-121.905,-124.514,-124.75,-122.469,...,-122.833,-119.708,-119.257,-119.651,-120.766,-119.004,-125.8,-123.55,-123.45,-126.069
geospatial_lon_max,-130.36,-120.857,-123.301,-123.969,-119.044,-122.82,-121.857,-124.485,-124.704,-122.396,...,-122.833,-119.708,-119.257,-119.651,-120.766,-119.004,-125.744,-123.55,-123.45,-126.013
time_coverage_start,1975-07-17T02:00:00Z,1980-10-07T18:00:00Z,1981-04-01T00:00:00Z,1981-04-01T19:00:00Z,1982-04-20T20:00:00Z,1982-07-23T16:00:00Z,1983-12-01T22:00:00Z,1984-03-27T23:00:00Z,1987-06-09T17:00:00Z,1987-06-17T17:00:00Z,...,2011-02-10T19:21:00Z,2011-06-10T16:42:00Z,2014-10-01T20:35:00Z,2015-03-18T14:37:00Z,2015-07-10T14:00:00Z,2017-09-24T16:00:00Z,1970-06-26T23:00:00Z,1990-11-27T20:50:00Z,1992-10-29T17:50:00Z,2012-10-31T00:19:00Z
time_coverage_end,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,...,2012-11-18T14:34:00Z,2012-09-30T19:12:00Z,2015-12-03T17:05:00Z,2017-04-02T04:16:00Z,2017-07-24T18:30:00Z,2018-11-20T17:30:00Z,1998-06-17T08:50:00Z,1991-04-30T08:50:00Z,1994-01-05T16:20:00Z,2013-04-28T21:45:00Z
institution,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,U.S. Army Corps of Engineers,University of Washington Applied Physics Labor...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Fisheries and Oceans Canada Marine Environment...,Environment and Climate Change Canada,Environment and Climate Change Canada,Fisheries and Oceans Canada
date_update,2023-06-03T17:45:21Z,202

In [46]:
# Clear the map and draw the search area defined above
m = folium.Map(location=[53.0, 0], zoom_start=4)
upper_left = [targeted_geospatial_lat_max, targeted_geospatial_lon_min]
upper_right = [targeted_geospatial_lat_max, targeted_geospatial_lon_max]
lower_right = [targeted_geospatial_lat_min, targeted_geospatial_lon_max]
lower_left = [targeted_geospatial_lat_min, targeted_geospatial_lon_min]
edges = [upper_left, upper_right, lower_right, lower_left]
polygon = folium.vector_layers.Polygon(locations=edges)
m.add_child(polygon)
m.fit_bounds(polygon.get_bounds())
# Plot the selected stations based on the subset selected above
for ind in subset2.index:
    legend='First measurement: '+subset2['time_coverage_start'][ind]+' Last measurement: '+subset2['last_date_observation'][ind]+' Platform code:'+subset2['platform_code'][ind]+' Parameters:'+subset2['parameters'][ind]
    folium.Marker(location=[subset2['last_latitude_observation'][ind],subset2['last_longitude_observation'][ind]],popup=legend).add_to(m)
# Show map
m
# Save map
# map.save('map_test.html')

### Highlight stations that are live

To highlight in the map the live stations, the criteria to be used is based on the last observation date

In [47]:
# If coastline filter not applied
subset2=subset1

In [48]:
# Criteria: last observation date within this week
date_format='%Y-%m-%dT%H:%M:%SZ'
dtheshold=datetime.timedelta(days=7)
dnow=datetime.datetime.strftime(datetime.datetime.now()-dtheshold,date_format)
condition_last_obs = subset1['last_date_observation']>dnow

In [49]:
subset3=subset2[condition_last_obs]
subset3.transpose()

,1361,1467,1529,1590,1652,1713,1774,1835,1897,1958,...,4433,4494,4555,4616,4677,4738,4799,4842,5181,9530
# product_id,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,...,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01,COP-GLOBAL-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_GLO_PHYBGCWA...
geospatial_lat_min,42.57,34.868,38.228,39.196,33.739,37.75,41.84,35.703,46.12,47.336,...,32.957,48.173,34.022,36.934,33.062,33.29,36.845,33.336,32.867,45.958
geospatial_lat_max,42.664,35.0,38.253,39.235,33.763,37.759,41.852,35.774,46.163,47.353,...,32.957,48.173,34.022,36.934,33.062,33.29,36.845,33.336,32.87,46.14
geospatial_lon_min,-130.537,-121.019,-123.325,-123.974,-119.08,-122.839,-124.386,-121.905,-124.514,-124.75,...,-117.279,-123.607,-118.578,-122.034,-117.314,-117.5,-121.825,-117.659,117.26,-131.07899
geospatial_lon_max,-130.36,-120.857,-123.301,-123.969,-119.044,-122.82,-124.38,-121.857,-124.485,-124.704,...,-117.279,-123.607,-118.578,-122.034,-117.314,-117.5,-121.825,-117.659,-117.257,-131.0
time_coverage_start,1975-07-17T02:00:00Z,1980-10-07T18:00:00Z,1981-04-01T00:00:00Z,1981-04-01T19:00:00Z,1982-04-20T20:00:00Z,1982-07-23T16:00:00Z,1983-09-01T00:00:00Z,1983-12-01T22:00:00Z,1984-03-27T23:00:00Z,1987-06-09T17:00:00Z,...,2019-12-06T17:30:00Z,2020-04-01T19:00:00Z,2020-08-21T17:00:00Z,2020-12-07T22:00:00Z,2022-08-30T21:26:00Z,2022-10-26T13:56:00Z,2023-04-20T20:00:00Z,2023-05-31T03:30:00Z,2005-01-01T00:30:00Z,1976-09-11T19:00:00Z
time_coverage_end,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,2023-05-31T23:50:00Z,...,2023-05-31T23:56:00Z,2023-05-31T23:00:00Z,2023-05-31T22:30:00Z,2023-05-31T23:00:00Z,2023-05-31T23:56:00Z,2023-05-31T23:00:00Z,2023-05-31T23:00:00Z,2023-05-31T23:00:00Z,2023-05-31T22:20:00Z,2023-05-31T23:50:00Z
institution,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,National Data Buoy Center (NDBC) ; National Oc...,...,U.S. Army Corps of Engineers,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Scripps Institution of Oceanography La Jolla (...,Unknown institution,Unknown institution,Unknown institution,Unknown institution,Scripps Institution of Oceanography La Jolla (...,National Data Buoy Center (NDBC) ; National Oc...
date_update,2023-06-03T17:45:21Z,2023-06-03T16:46:04Z,2023-06-03T17:13:57Z,2023-06-03T17:50:04Z,2023-06-03T18:17

In [50]:
# Plot the selected stations based on the subset selected above
for ind in subset3.index:
    legend='First measurement: '+subset3['time_coverage_start'][ind]+' Last measurement: '+subset3['last_date_observation'][ind]+' Platform code:'+subset3['platform_code'][ind]+' Parameters:'+subset3['parameters'][ind]
    folium.Marker(location=[subset3['last_latitude_observation'][ind],subset3['last_longitude_observation'][ind]],
                  popup=legend,
                    icon=folium.Icon(color='green')).add_to(m)
# Show map
m
# Save map
#m.save('CMEMS_world_waves_Hm0_Tm02.html')

## Data download

In [ ]:
# Save subset to csv file
# subset.to_csv('subsetOffiles.csv')

In [51]:
output_directory = os.getcwd()+'\\'+'data' #default to subfolder "data" on current working directory
output_directory

'u:\\PSAN\\data'

In [52]:
treshold = 8 #Please set a file number maxima to download -- to be removed

In [55]:
with ftputil.FTPHost(dataset['host'], user, password) as ftp_host:  # connect to CMEMS FTP
    for i in range(0, len(subset2)):
        if i<treshold:
            filepath = subset2.iloc[i]['file_name'].split(dataset['host'])[1]
            ncdf_file_name = filepath.split('/')[-1]
            if ftp_host.path.isfile(filepath):
                print('.....Downloading ' + ncdf_file_name)
                cwd = os.getcwd()
                os.chdir(output_directory)
                try:
                    ftp_host.download(filepath, ncdf_file_name)  # remote, local
                    print('Done!')
                except Exception as e:
                    print('error!: FTP download failed ...')
                os.chdir(cwd)

.....Downloading GL_TS_MO_46002.nc
Done!
.....Downloading GL_TS_MO_46011.nc
Done!
.....Downloading GL_TS_MO_46013.nc
Done!
.....Downloading GL_TS_MO_46014.nc
Done!
.....Downloading GL_TS_MO_46025.nc
Done!
.....Downloading GL_TS_MO_46026.nc
Done!
.....Downloading GL_TS_MO_46027.nc
Done!
.....Downloading GL_TS_MO_46028.nc
Done!
